In [1]:
import csv
import sys
sys.path.append('../astroML-coral/Tools')
sys.path.append('../astroML-coral/Data')
import datascrub as scrub
import mldataconfig as dtc
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import pprint
import progressbar
import gc

In [3]:
scrub.exofinder('Data/kepler_lightcurves_Q03_long.txt', 'bigtest.csv', False)

<class 'list'>
Kepler pipeline data converted
Successfuly pulled system data from:  Data/kepler_lightcurves_Q03_long.txt


In [4]:
kepid = []
url = []
desig = []
with open('bigtest.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        kepid.append(row[0])
        url.append(row[1])
        desig.append(row[2])
full = zip(kepid, url, desig)
full = [list(ele) for ele in full]
training = full[1:100000]
test = full[100000:]

In [5]:
kepid = []
url = []
desig = []
with open('bigtest.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        kepid.append(row[0])
        url.append(row[1])
        desig.append(row[2])
full = zip(kepid, url, desig)
full = [list(ele) for ele in full]
training = full[1:100000]
test = full[100000:]

print(test[0])


['8957928', 'https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:Kepler/url/missions/kepler/lightcurves/0089/008957928//kplr008957928-2009350155506_llc.fits', '0']


In [6]:
test, training, full = dtc.datasplit('bigtest.csv', 100000)
print(test[0])

['757450', 'https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:Kepler/url/missions/kepler/lightcurves/0007/000757450//kplr000757450-2009350155506_llc.fits', '1']


In [12]:
print(len(test))

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def getfluxes(dat, output):
    with open(output,'ab') as f:
        bar = progressbar.ProgressBar(maxval=len(dat)).start()
        for idx, i in enumerate(dat):
            ret = []
            k2_bjds = []
            pdcsap_fluxes = []
            fits_file = i[1]
            with fits.open(fits_file, mode="readonly") as hdulist:    
                k2_bjds = hdulist[1].data['TIME']
                #sap_fluxes = hdulist[1].data['SAP_FLUX']
                pdcsap_fluxes = hdulist[1].data['PDCSAP_FLUX']
            hdulist.close()
            del hdulist

            flux = np.transpose(pdcsap_fluxes)    
            ret = np.insert(pdcsap_fluxes, 0, i[0], axis=0)
            ret = np.insert(ret, 1, i[2], axis=0)
            #print('Finished: '+ i[0])
            np.savetxt(f, ret.reshape(1, ret.shape[0]), delimiter=",")
            del ret, k2_bjds, pdcsap_fluxes, fits_file, flux, i
            bar.update(idx)
        gc.collect()
        
    

99999


In [8]:
dat = list(chunks(test, 5000))

#for idx, i in enumearate(dat):
#    name = 'test%s.csv' % idx
 #   print(name)
  #  getfluxes(i, name)
    
dtc.getfluxes(dat[0], 'data1.csv')

 99% (4999 of 5000) |################### | Elapsed Time: 0:29:35 ETA:   0:00:00

In [13]:
getfluxes(dat[1], 'data2.csv')

  0% (31 of 5000) |                      | Elapsed Time: 0:00:10 ETA:   0:42:44

KeyboardInterrupt: 

In [54]:
dtc.getfluxes(dat[2], 'data3.csv')

 18% (911 of 5000) |###                  | Elapsed Time: 0:07:58 ETA:   0:34:36

KeyboardInterrupt: 